In [ ]:
from selenium import webdriver
import bs4
from time import sleep
import pandas as pd
import re

In [ ]:
title = [] # 법률 상담목
body = [] # 질의 응답 내용

In [ ]:
def crawling(url, pages):
    
    driver = webdriver.Chrome('C:\Web_Driver\chromedriver.exe')#웹 드라이버가 저장되는 경로입니다.
    driver.implicitly_wait(1)
    current_page = url
    
    driver.get(current_page)
    
    sleep(3)

    html = driver.page_source
    soup = bs4.BeautifulSoup(html, 'html.parser')

    cnt = 0 

    for k in range(2, pages):
        for i in range(4, 24):
            try: # 해당 목록이 끝날 경우 
                driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/form[1]/fieldset/table/tbody/tr[{}]/td[2]/a'.format(i)).click()
            except:
                break

            driver.get(driver.current_url)

            html = driver.page_source
            soup = bs4.BeautifulSoup(html, 'html.parser')

            title.append(driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[3]/div/div[1]/div[1]/h3/a').text)
            body.append(driver.find_element_by_class_name('boardReadBody').text)

            print(title[-1])

            cnt += 1

        try:
            driver.get(current_page + '&page={}'.format(k))

            driver.get(driver.current_url)
        except:
            
            break
    print('[+] 크롤링을 마쳤습니다. 총 파싱한 게시글 수 : {} [+]'.format(cnt))        
    return title, body

In [ ]:
urls = [
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna01',10], # 가사소송
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna02',12], # 강제집행
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna03',7], # 주택임대차
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna04',4], # 상가임대차
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna05',8], # 근로관계
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna06',4], # 파산회생
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna07',42], # 민사소송
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna08',10], # 상사소송
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna09',20], # 형사소송
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna10',10], # 행정소송
    ['http://www.freelawfirm.co.kr/index.php?mid=lawqna11',8], # 헌법소송
]

def combine(url,num):
    global title
    global body
    
    temp1,temp2 = crawling(url, num)
    
    title += temp1
    body += temp2
    
    print()

In [ ]:
for a,b in urls:
    combine(a,b)

In [ ]:
Q = []
A = []

In [ ]:
title

In [ ]:
body[0]

In [ ]:
q = re.compile('\[결론\].{5,}\\n     이')

In [ ]:
for i in range(len(title)):
    try:
        if q.findall(body[i])[0]:
            print(title[i])
            print(q.findall(body[i])[0])
            print()
            Q.append(title[i])
            A.append(q.findall(body[i])[0])
    except:
        print()

In [ ]:
Q = list(map(lambda x: x[7:], Q))

A = list(map(lambda x: x[5:-7], A))

In [ ]:
len(Q)

In [ ]:
len(A)

In [ ]:
chatbotdata = pd.DataFrame({"Q":Q, "A":A, "label":[3]*246440})

In [ ]:
chatbotdata.to_csv('ChatbotData.csv') # save